<a href="https://colab.research.google.com/github/sriharikrishna/siamcse21/blob/main/scalar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises
1. Write your own scalar function
2. Use jax.grad() jax.jvp() jax.vjp()
3. Change the seed values that have been provided



### 1. Primal Function
1. Evaluates the scalar function for an arbitary input vector

In [ ]:
import jax
from jax import random
import jax.numpy as jnp

def fun(x):
    """
    Input: x vector of values
    Output: Result of a scalar function
    """
    #Enter your computation here OR use the following 
    #Speelpenning function
    result = jnp.prod(x**2)

    return result

### 2. Primal Driver
1. Calls the scalar function. 
2. We have created a random array as its input.

In [ ]:
def fun_driver(n):
    """
    Input: n array length
    Output: Result of scalar function
    """
    #create a random array
    key = random.PRNGKey(0)
    val = random.normal(key, (n,), jnp.float64)

    #create a array of constant elememnts
    #val = jnp.full(n, 0.5)

    #compute the result
    result = fun(val)

    print("The input is", val)
    print("The result is ", result)

In [ ]:
fun_driver(10)

### 3. First look at derivatives: `jax.grad()`
1. https://jax.readthedocs.io/en/latest/jax.html#jax.grad
2. Computes $v\cdot J$ for a function that computes a scalar value ($R^n \rightarrow R$).
3. The seed is internally set to `1.0`
   (the shape of the seed must match the primal output)

In [ ]:
#Create a function that computes the derivatives
grad_fun = jax.grad(fun)

def grad_driver(n):
    """
    Input: n array length
    Output: Derivatives of the scalar function
    """
    
    #create a random array
    key = random.PRNGKey(0)
    val = random.normal(key, (n,), jnp.float64)

    #create a array of constant elememnts
    #val = jnp.full(n, 0.5)
    
    #compute the result
    result = grad_fun(val)
    print("The input is", val)
    print("The grad result is ", result)

In [ ]:
grad_driver(10)

### 4. Forward mode using: `jax.jvp()`
1. https://jax.readthedocs.io/en/latest/jax.html#jax.jvp
2. Computes $J\cdot v$ for a function $R^n \rightarrow R^m$. 
3. `jax.jvp()` requires the input value for the primal function, which is the point where the derivatives are computed.
4. `jax.jvp()` requires a seed $v$. 
   (For forward mode the shape of the seed must match the primal input.) 
5. The code below obtains the entire Jacobian by calling `jax.jvp()` multiple times. It sets the input value to an arbitary value and sets the seed to the Cartesian basis vectors.

In [ ]:
def jvp_driver(n):
    """
    Input: n array length
    Output: Derivatives of the scalar function
    """
    #create a random array
    key = random.PRNGKey(0)
    val = random.normal(key, (n,), jnp.float64)

    #create a array of constant elememnts
    #val = jnp.full(n, 0.5)

    tangents = jnp.empty([0])

    #compute the derivatives. It takes n calls!
    seed = jnp.eye(n)  

    #uncomment one of the statements below to use some other seed
    #seed = random.normal(key, (n,n,), jnp.float64)
    #seed = seed * 0.5
    #seed = jnp.full((n,n,), 2.0)

    for i in range(n):
        # jax.jvp must be called once for each input and seed value
        primal_output, res = jax.jvp(fun, (val,), (seed[i],))
        tangents = jnp.append(tangents, res)
    
    print("The input is", val)
    print("The jax.jvp result is ", tangents)

In [ ]:
jvp_driver(10)

### 5. Reverse mode using: `jax.vjp()`
1. https://jax.readthedocs.io/en/latest/jax.html#jax.vjp
2. Uses reverse mode, returns a function that computes the adjoints for a funtion $R^n \rightarrow R^m$.
3. `jax.vjp()` requires the input value for the primal function, which is the point where the derivatives are computed.
4. `fun_vjp` requires a seed $v$. For reverse mode the shape of the seed must match the primal output. We have chosen `1.0`. 
5. Exercise: See how values change as the seed changes.


In [ ]:
def vjp_driver(n):    
    """
    Input: n array length
    Output: Derivatives of the scalar function
    """
    #create a random array
    key = random.PRNGKey(0)
    val = random.normal(key, (n,), jnp.float64)

    #create a array of constant elememnts
    #val = jnp.full(n, 0.5)

    #jax.vjp must be called once for each input value
    primals, fun_vjp = jax.vjp(fun, val)
    
    #compute the derivatives by calling fun_vjp() for the seed value
    #try using a different seed value.
    seed = 1.0
    adj_val = fun_vjp(seed)

    print("The input is", val)
    print("The jax.vjp result is ", adj_val[0])

In [ ]:
vjp_driver(10)